In [1]:
import pymssql
import pyodbc
import pandas as pd
from datetime import datetime as dt

C:\Users\bustossanchez.9\AppData\Roaming\Python\Python37\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
# SERVER = 'TPCCP-DB09\SCNEAR'
#USERNAME = 'ScAdUser'
#PASSWORD = '5c0r3Card*77'
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                      'Server= TPCCP-DB09\SCNEAR;'
                      'Database=Infinite;'
                      'Trusted_Connection=yes;')

QUERY = '''SELECT [SURVEY_RETURNED_DATE] as 'Date'
      ,[AGENT_NAME] as 'Agent'
      ,[EMAIL]
      , CASE WHEN [TICKET_TYPE_SIMPLIFIED] like 'Instacart Customers' OR [USER_CHANNEL] like 'Customer' THEN 'Customers' ELSE 'Shoppers' END Type
	  ,CASE WHEN [CHANNEL_SIMPLIFIED] like 'Instacart Generated' OR [CHANNEL_SIMPLIFIED] like 'Ratings Flow' OR [CONTACT_CHANNEL] like 'Instacart Generated' OR [CONTACT_CHANNEL] like 'Ratings Flow' THEN 'Email'
			WHEN [CHANNEL_SIMPLIFIED] like 'Phone' OR [CONTACT_CHANNEL] like 'Phone' THEN 'Phone'
			WHEN [CHANNEL_SIMPLIFIED] like 'Chat' OR [CONTACT_CHANNEL] like 'Chat' THEN 'Chat'
			ELSE 'Email' END Channel  	  
	  ,[SUBCASE_TYPE_LEVEL2] as 'CR'
      ,CASE WHEN [MENU_PATH_NAME] like 'My Missing Order' OR [MENU_PATH_NAME] like 'Missing Order' OR [MENU_PATH_NAME] like 'Customer Support/Missing Order' OR [MENU_PATH_NAME] like '' OR [MENU_PATH_NAME] is null THEN 'FDO' 
	  WHEN [MENU_PATH_NAME] like 'Customer Support/Costco Order' OR [MENU_PATH_NAME] like 'Costco Order' OR [MENU_PATH_NAME] like 'Active Costco Order' then 'COSTCO' ELSE 'T1' END Menu_Path
	  , CASE WHEN [Rating] like 'good' THEN 'Good' ELSE 'Bad' END CSAT
	  ,nullif(RATING_COMMENT,'') as 'Feedback'
      ,[TOUCH]
      ,[WAREHOUSE]

	 
  FROM [Instacart].[dbo].[tbSFTP_CSAT] with(nolock)
  WHERE ([SURVEY_RETURNED_DATE] >= '2022-01-01') and ([CONTACT_CENTER] like 'Teleperformance - Lima')
  ORDER BY [Date] DESC'''
################### DONT TOUCH ###################
#connection = pymssql.connect( server = SERVER, user = USERNAME, password = PASSWORD, database = 'Instacart')

#cursor = connection.cursor(as_dict=True)
#cursor.execute(QUERY)
#data = cursor.fetchall()

BDInfinite = pd.read_sql(sql=QUERY,con=conn)

df=BDInfinite
df

,Date,Agent,EMAIL,Type,Channel,CR,Menu_Path,CSAT,Feedback,TOUCH,WAREHOUSE
0,2023-06-21,Silvia Ojeda Vizcarra,silvia.ojedavizcarra@teleperformance.com,Customers,Chat,Entire Delivery Missing,FDO,Bad,None,1,Best Buy
1,2023-06-21,Ingrid Pilar Rodriguez Pastor,ingrid.rodriguezpastor@teleperformance.co,Customers,Chat,Entire Delivery Missing,FDO,Good,None,2,Safeway
2,2023-06-21,Carlos Alejandro Archila Ruiz,carlos.archilaruiz@teleperformance.com,Customers,Chat,Entire Delivery Missing,FDO,Good,None,2,GIANT
3,2023-06-21,Arnold Gongora Rios,arnold.gongorarios@teleperformance.com,Customers,Chat,Entire Delivery Missing,FDO,Bad,None,2,Wegmans
4,2023-06-21,Paolo Velarde Jimenez,paolo.velardejimenez@teleperformance.co,Customers,Phone,Not Enough Information,T1,Good,None,1,None
...,...,...,...,...,...,...,...,...,...,...,...
789558,2022-01-01,Jairo Alfredo Silva Vilela,jairo.silvavilela@teleperformance.com,Customers,Chat,Entire Delivery Missing,FDO,Good,None,1,Walgreens
789559,2022-01-01,Miguel Ramirez Jara,miguel.ramirezjara@teleperformance.com,Customers,Phone,Change Delivery Instructions,T1,Good,None,1,Food Lion Now
789560,2022-01-01,Marcos Jose Marcano Pena,marcos.marcanopena@teleperformance.com,Customers,Phone,Where Is My Order,T1,Good,None,1,Food Lion
789561,2022-01-01,Gina Babilon Babilon,gina.babilonbabilon@teleperformance.com,Customers,Chat,Entire Delivery Missing,FDO,Good,None,1,Shoppers Drug Mart


In [3]:
df["len"]=df["Feedback"].apply(lambda x: len(str(x)))
df=df[df["len"] >4]
df=df.drop("len", axis=1)
df=df.set_index("Date")
df.to_csv(r"C:\Users\bustossanchez.9\OneDrive - Teleperformance\Procesos\Instacard\RawData\Data_TM.csv")

In [4]:
import nltk
from nltk.probability import FreqDist
import pandas as pd
import datetime as dt
import numpy as np
from collections import OrderedDict
pd.options.mode.chained_assignment = None
import unicodedata
import regex as re
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence 

In [5]:
#LEER DATA
cols=["Date", "Type", "Channel", "CSAT", "Feedback"]
df=pd.read_csv(r"C:\Users\bustossanchez.9\OneDrive - Teleperformance\Procesos\Instacard\RawData\Data_TM.csv",usecols=cols)
#OBTENER EL MES
df["Date"]=pd.to_datetime(df['Date'])
df["Month"]=df.Date.dt.month_name()
df["Year"]=df.Date.dt.year
df=df.drop("Date", axis=1)
#Crea columna con el CSAT/lob y borra las otras columnas:
df['index_2'] = df[["CSAT", "Channel", "Type"]].apply(lambda x:'/'.join(x), axis = 1)
df = df.drop(["CSAT", "Channel", "Type"], axis = 1)
#Crea un index doble, por semana y CSAT/Lob. 
df = df.set_index(["Year","Month", "index_2"]).sort_index(ascending = [True, True]).astype(str)
#Agrupa para que solo se tengan por cada semana, 1 fila de CSAT/lob y aplica transform para unir todo el texto que aplica en una celda. 
df= df.groupby(["Year","Month", "index_2"]).transform(lambda x : ' '.join(x)).drop_duplicates(keep = 'first')
df["Feedback"] = df["Feedback"].apply(lambda x: str(x).lower())
df['Feedback'] = df['Feedback'].apply(lambda x: x.encode('utf-8').decode('ascii', 'ignore'))
df["words"] = df["Feedback"].apply(lambda x: list(nltk.word_tokenize(x)))
common_words = ["nbsp",'<', 'br', '>',"!",".",",", "...",":", "n't","(",")","wasn't","", "nan"]

#Remover stopwords y puntuacion
def remove_stopwords(listica):
    from nltk.corpus import stopwords                   
    stop_words = set(stopwords.words("english"))        #Getting a list of stopwords
    removedstopwords = [w for w in listica if not w in stop_words and w not in common_words and len(w) > 2]  # code to remove stop words
    return removedstopwords

df["words"] = df["words"].apply(lambda x: remove_stopwords(x))

#Lemmatizacion: 
def lemmatizacion(listica2):
    WNlemma = nltk.WordNetLemmatizer()       #Getting a list of stopwords
    lemmatized_output = [WNlemma.lemmatize(w, pos = "v") for w in listica2]  # code to remove stop words
    return lemmatized_output

df["lemma"]=df["words"].apply(lambda x: lemmatizacion(x))
df=df.drop(["words", "Feedback"], axis=1)

df1=df.copy()
df2=df.copy()
df3=df.copy()

#### WORDS ####
df1["frecwords"]=df1["lemma"].apply(lambda x: FreqDist(x))
df1.reset_index(inplace=True)
df1_temp =pd.DataFrame(pd.DataFrame(df1['frecwords'].values.tolist()).stack().reset_index(level=1))
df1_temp.columns = ['Word', 'Frequency']
df1 = df1.join(df1_temp)
df1[["CSAT", "Channel", "Type"]]=df1['index_2'].str.split('/', 2, expand=True)
df1.Word=df1.Word.apply(lambda x: x.strip())
df1.drop(["frecwords","lemma","index_2"], axis=1,inplace=True)
df1=df1[df1["Frequency"]>1]
df1 = df1[["Year",'Month','CSAT','Channel','Type','Word','Frequency']]
df1 = df1.rename(columns={'CSAT':'Csat','Frequency':'Frecuency'})
df1.to_csv(r"C:\Users\bustossanchez.9\OneDrive - Teleperformance\Procesos\Instacard\DataPBI\Words.csv",index=False)

In [6]:
##### BIGRAMS ####

def common_bigramss(listica3):
    md_bigrams = list(nltk.bigrams(listica3))
    threshold = 2
    filtered_bigrams = [bigram for bigram in md_bigrams if len(bigram[0])>threshold and len(bigram[1])>threshold]
    filtered_bigram_dist = FreqDist(filtered_bigrams)
    return filtered_bigram_dist

df2["bigrams"]=df2["lemma"].apply(lambda x: common_bigramss(x))
df2.reset_index(inplace=True)
df2_temp =pd.DataFrame(pd.DataFrame(df2['bigrams'].values.tolist()).stack().reset_index(level=1))
df2_temp.columns = ['Bigram', 'Frequency']
df2_temp[["Word1","Word2"]]=pd.DataFrame(df2_temp['Bigram'].tolist(), index=df2_temp.index)
df2 = df2.join(df2_temp)
df2[["CSAT", "Channel", "Type"]]=df2['index_2'].str.split('/', 2, expand=True)
df2.Word1=df2.Word1.apply(lambda x: str(x))
df2.Word1=df2.Word1.apply(lambda x: x.strip())
df2.Word2=df2.Word2.apply(lambda x: str(x))
df2.Word2=df2.Word2.apply(lambda x: x.strip())
df2.drop(["bigrams","Bigram","lemma","index_2"], axis=1,inplace=True)
df2=df2[df2["Frequency"]>1]
df2 = df2.rename(columns={'CSAT':'Csat','Frequency':'Frecuency'})
df2 = df2[["Year",'Month','Csat','Channel','Type','Word1','Word2','Frecuency']]
df2.head()

df2.to_csv(r"C:\Users\bustossanchez.9\OneDrive - Teleperformance\Procesos\Instacard\DataPBI\Bigrams.csv",index=False)

In [7]:
###### Lista de Palabras Unicas ################################################

df=pd.read_csv(r"C:\Users\bustossanchez.9\OneDrive - Teleperformance\Procesos\Instacard\DataPBI\Words.csv")
words=df["Word"]
unique_words=words.unique()
key_unique_words = pd.DataFrame(unique_words)
key_unique_words .rename(columns = {0:'Word'}, inplace = True)
key_unique_words["Word"]= key_unique_words["Word"].map(str)
key_unique_words=key_unique_words["Word"].apply(lambda x: x.strip())
key_unique_words.to_csv(r"C:\Users\bustossanchez.9\OneDrive - Teleperformance\Procesos\Instacard\DataPBI\Key_Unique_Words.csv")

In [8]:
#### TRIGRAMS #####

def common_trigrams(listica3):
    md_trigrams = list(nltk.trigrams(listica3))
    filtered_bigram_dist = FreqDist(md_trigrams)
    return filtered_bigram_dist

df3["trigrams"]=df3["lemma"].apply(lambda x: common_trigrams(x))
df3.reset_index(inplace=True)
df3_temp =pd.DataFrame(pd.DataFrame(df3['trigrams'].values.tolist()).stack().reset_index(level=1))
df3_temp.columns = ['trigram', 'Frequency']
df3_temp[["Word1","Word2","Word3"]]=pd.DataFrame(df3_temp['trigram'].tolist(), index=df3_temp.index)
df3 = df3.join(df3_temp)
df3[["CSAT", "Channel", "Type"]]=df3['index_2'].str.split('/', 2, expand=True)
df3.Word1=df3.Word1.apply(lambda x: str(x).strip())
df3.Word2=df3.Word2.apply(lambda x: str(x).strip())
df3.Word3=df3.Word3.apply(lambda x: str(x).strip())
df3.drop(["trigrams","trigram","lemma","index_2"], axis=1,inplace=True)
df3=df3[df3["Frequency"]>1]
df3 = df3.rename(columns={'CSAT':'Csat','Frequency':'Frecuency'})
df3 = df3[["Year",'Month','Csat','Channel','Type','Word1','Word2',"Word3",'Frecuency']]
df3.to_csv(r"C:\Users\bustossanchez.9\OneDrive - Teleperformance\Procesos\Instacard\DataPBI\Trigrams.csv",index=False)

In [9]:
import nltk
from nltk.probability import FreqDist
import pandas as pd
import datetime as dt
import numpy as np
from collections import OrderedDict
pd.options.mode.chained_assignment = None
import unicodedata
import regex as re
from nltk.stem import WordNetLemmatizer
from datetime import datetime as dt

In [10]:
#LEER DATA TRENDS
df=pd.read_csv(r"C:\Users\bustossanchez.9\OneDrive - Teleperformance\Procesos\Instacard\RawData\Data_TM.csv")

In [11]:
############### LIMPIEZA DE TEXTO EN FEEDBACK ##########################
#Lowercase: 
df["Feedback"] = df["Feedback"].apply(lambda x: str(x).lower())

#Se aplica la funcion a la columna
df['Feedback'] = df['Feedback'].apply(lambda x: x.encode('utf-8').decode('ascii', 'ignore'))


#Remover puntuacion
df['Feedback'] = df['Feedback'].apply(lambda x: x.replace(".", ""))
df['Feedback'] = df['Feedback'].apply(lambda x: x.replace(",", ""))
df['Feedback'] = df['Feedback'].apply(lambda x: x.replace("!", ""))
df=df.set_index("Date")

# LISTA Numero de palabras:
df["words"] = df["Feedback"].apply(lambda x: list(nltk.word_tokenize(x)))

#Remover stopwords y puntuacion
def remove_stopwords(listica):
    from nltk.corpus import stopwords                   
    stop_words = set(stopwords.words("english"))        #Getting a list of stopwords
    removedstopwords = [w for w in listica if not w in stop_words]  # code to remove stop words
    return removedstopwords

df["words"] = df["words"].apply(lambda x: remove_stopwords(x))

#Lemmatizacion: 
def lemmatizacion(listica2):
    WNlemma = nltk.WordNetLemmatizer()       #Getting a list of stopwords
    lemmatized_output = [WNlemma.lemmatize(w, pos = "v") for w in listica2]  # code to remove stop words
    return lemmatized_output

df["lemma"]=df["words"].apply(lambda x: lemmatizacion(x))

df=df.drop(["words"], axis=1)
def common_bigramss(listica3):
    #lista_nueva = listica3.split(',') # Volvemos el string una lista cuyos valores estan separados por ','
    md_bigrams = list(nltk.bigrams(listica3))
    #threshold = 2
    #filtered_bigrams = [bigram for bigram in md_bigrams if len(bigram[0])>threshold and len(bigram[1])>threshold]
    #filtered_bigram_dist = FreqDist(filtered_bigrams)
    return md_bigrams
df["bigrams"]=df["lemma"].apply(lambda x: common_bigramss(x))
df=df.drop("lemma", axis=1)
df["bigrams"]=df["bigrams"].map(str)
df["bigrams"]=df["bigrams"].apply(lambda x: x.replace("'",""))
df["bigrams"]=df["bigrams"].apply(lambda x: x.replace(" ",""))

df1=df
df2=df
df3=df

In [12]:
#####CATEGORIZATION AGENT################
Categories_agent = {"Contact Issues": ["(end,chat)","(leave,chat)","(chat,end)","(chat,without)","(take,time)","(timely,manner)","(time,get)","(put,hold)","(disconnect,chat)","(speak,supervisor)","(close,chat)","(ask,speak)","(agent,end)","(supervisor,call)","(speak,manager)","(let,speak)","(agent,rude)"],
                    "Customer Service": ["(helpful,thank)","(great,job)","(thank,help)","(service,rep)","(friendly,helpful)","(good,job)","(service,representative)","(support,person)","(service,agent)","(service,excellent)","(person,help)","(great,experience)","(agent,helpful)","(job,thank)","(support,agent)","(great,work)","(good,work)","(agent,great)","(agent,help)","(support,rep)","(great,agent)","(support,people)","(agent,good)"],
                    "Issue Resolution":["(resolve,issue)","(answer,question)","(issue,resolve)","(solve,problem)","(solve,issue)","(never,receive)","(resolve,problem)","(never,get)","(issue,quickly)","(fix,problem)","(fix,issue)","(go,beyond)","(didnt,get)","(problem,solve)","(quick,response)","(problem,resolve)","(someone,else)","(help,resolve)","(could,get)","(let,know)","(issue,thank)","(get,help)","(still,wait)","(help,issue)","(care,issue)","(could,help)","(able,help)","(get,answer)","(try,get)","(question,answer)","(keep,get)","(try,help)","(resolve,quickly)","(didnt,help)","(issue,still)","(help,get)","(address,issue)","(able,resolve)","(still,receive)","(help,solve)","(get,issue)","(could,find)","(still,resolve)","(handle,issue)","(ask,question)","(need,help)","(quick,easy)","(help,problem)","(get,resolve)","(person,speak)","(get,response)","(help,need)","(nothing,resolve)","(issue,never)","(issue,solve)","(still,get)","(never,resolve)","(resolve,thank)","(issue,take)","(issue,great)","(try,resolve)","(resolution,problem)","(issue,get)","(question,ask)","(receive,response)","(without,help)","(could,resolve)","(really,help)","(help,fix)","(problem,fix)","(couldnt,help)","(time,thank)","(give,answer)","(problem,still)","(problem,get)","(find,solution)","(didnt,resolve)","(get,resolution)","(resolve,situation)","(wasnt,resolve)","(run,around)","(solution,problem)","(get,problem)","(really,need)","(time,issue)","(unable,help)","(without,resolve)","(actually,help)","(solve,problems)","(still,unresolved)","(back,forth)","(still,need)","(explain,issue)","(help,understand)","(help,customer)","(help,find)","(issue,fix)","(issue,unresolved)","(answer,phone)","(resolve,concern)","(give,information)","(resolve,still)","(unable,resolve)","(agent,say)","(information,need)","(know,answer)","(issue,give)","(issue,could)","(agent,tell)","(need,fix)","(problem,never)","(issue,need)","(issue,agent)","(issue,please)","(resolve,anything)","(issue,first)","(get,fix)","(answer,give)","(issue,support)","(resolve,didnt)","(agent,could)","(remain,unresolved)","(issue,dont)","(resolve,question)","(provide,information)","(still,issue)","(everything,handle)","(support,issue)","(actually,resolve)"],
                    "English Skills": ["(could,understand)","(understand,situation)","(hard,understand)","(get,someone)","(understand,say)","(difficult,understand)","(speak,english)","(hire,people)","(hard,time)","(agent,speak)","(time,understand)","(could,barely)","(try,understand)","(better,communication)","(language,barrier)","(people,speak)","(barely,understand)","(english,speak)","(understand,english)","(english,language)","(barely,speak)"],
                    "Listen/Understand":["(understand,issue)","(understand,problem)","(pay,attention)","(dont,understand)","(didnt,understand)","(listen,issue)","(agent,understand)","(issue,understand)","(listen,customer)","(understand,try)","(couldnt,understand)"]
                    }

################### FUNCION PARA CATEGORIZAR ############################################
def get_category(x):
    key_list = []
    for key in Categories_agent.keys():
        for value in Categories_agent[key]:
            if value in x:
                if key not in key_list:
                    key_list.append(key)
    return(key_list)

################### FUNCION PARA RETORNAR EL BIGRAMA ##################################3

def get_bigram_key(x):
    key_list = []
    for value in Categories_agent[x["Category"]]:
        if value in x["bigrams"]:
            if value not in key_list:
                key_list.append(value)
    return(key_list)

# Se aplica la funcion al df:
df1["Category"]=df1["bigrams"].apply(lambda x: get_category(x))
df1["Category"]=df1["Category"].map(str)

df1=df1[df1["Category"]!="[]"]
df1["Category"]=df1["Category"].map(str).apply(lambda x: x.replace("[",""))
df1["Category"]=df1["Category"].map(str).apply(lambda x: x.replace("]",""))
df1["Category"]=df1["Category"].map(str).apply(lambda x: x.replace("'",""))
df1["Category"]=df1["Category"].map(str).apply(lambda x: x.strip())
df1["Category"]=df1["Category"].map(str)
df1.Category = df1.Category.str.split(",")
df1 = df1.explode('Category')
df1["Category"]=df1["Category"].map(str).apply(lambda x: x.strip())
#Obtener bigramas por categoria: 
df1["Category2"]=df1.apply(get_bigram_key, axis=1)
df1["Category2"]=df1["Category2"].map(str).apply(lambda x: x.replace("[",""))
df1["Category2"]=df1["Category2"].map(str).apply(lambda x: x.replace("]",""))
df1["Category2"]=df1["Category2"].map(str).apply(lambda x: x.replace("'",""))
df1["Category2"]=df1["Category2"].map(str).apply(lambda x: x.replace("), (","-"))
df1["Category2"]=df1["Category2"].map(str)
df1.Category2 = df1.Category2.str.split("-")
df1 = df1.explode('Category2')
df1["Category2"]=df1["Category2"].map(str).apply(lambda x: x.replace("(",""))
df1["Category2"]=df1["Category2"].map(str).apply(lambda x: x.replace(")",""))
df1["Category2"]=df1["Category2"].map(str).apply(lambda x: x.replace(","," "))
df1["Category2"]=df1["Category2"].map(str).apply(lambda x: x.strip())
df1=df1.drop("bigrams", axis=1)
df1["Category1"]="Agent"
df1.to_csv(r"C:\Users\bustossanchez.9\OneDrive - Teleperformance\Procesos\Instacard\RawData\Categorization_Agent.csv")


#####CATEGORIZATION BRAND ################
Categories_brand = {"Account":["(cancel,account)","(email,address)","(close,account)","(account,still)","(account,deactivate)","(account,get)","(account,tell)","(get,account)","(delete,account)","(deactivate,account)","(new,account)","(reset,password)","(account,please)","(account,hack)","(help,account)","(account,issue)"],
                    "App": ["(use,app)","(app,say)","(app,work)","(delete,app)","(app,order)","(issue,app)","(app,issue)","(app,need)","(fix,app)"],
                    "Billing/Payment":["(credit,card)","(get,refund)","(credit,account)","(give,credit)","(get,credit)","(receive,refund)","(gift,card)","(money,back)","(bank,account)","(receive,credit)","(debit,card)","(offer,credit)","(full,refund)","(refund,credit)","(refund,money)","(refund,order)","(order,charge)","(want,refund)","(get,charge)","(promo,code)","(get,money)","(charge,card)","(issue,credit)","(charge,twice)","(credit,order)","(issue,refund)","(give,refund)","(back,account)","(take,money)","(need,refund)","(account,credit)","(want,money)","(want,credit)","(credit,back)","(order,refund)","(need,change)","(payment,method)","(charge,account)","(offer,refund)","(pay,order)","(order,credit)","(money,refund)","(pay,extra)","(refund,back)","(get,full)","(need,money)","(back,card)","(credit,refund)","(give,money)","(refund,issue)","(money,account)","(get,cancel)","(steal,money)","(refund,account)","(full,amount)","(time,money)","(issue,account)","(money,order)","(get,pay)","(pay,full)","(refund,cancel)","(need,support)","(refund,give)","(card,decline)","(receive,money)","(refund,full)"],
                    "Contact Issues": ["(call,back)","(get,back)","(come,back)","(second,time)","(receive,email)","(send,email)","(call,customer)","(response,time)","(contact,customer)","(contact,support)","(take,minutes)","(hear,back)","(email,say)","(call,support)","(time,take)","(wait,long)","(back,get)","(contact,driver)","(try,contact)","(get,touch)","(amount,time)","(wait,days)","(tell,contact)","(text,message)","(able,contact)","(time,contact)","(time,resolve)","(way,contact)","(never,call)","(order,contact)","(need,call)","(get,contact)","(answer,call)","(need,contact)","(someone,contact)","(contact,someone)"],
                    "Customer Service":["(great,customer)","(great,service)","(excellent,customer)","(service,thank)","(excellent,service)","(customer,support)","(great,help)","(super,helpful)","(good,customer)","(great,support)","(excellent,support)","(bad,experience)","(several,time)","(terrible,service)","(terrible,customer)","(horrible,customer)","(support,thank)","(best,customer)","(good,support)","(worst,customer)","(horrible,experience)","(poor,service)","(thank,support)","(support,team)","(different,people)","(bad,service)","(awesome,thank)","(keep,good)","(need,better)","(support,staff)","(service,suck)","(tech,support)","(support,always)","(support,get)","(care,customers)","(better,customer)","(better,service)","(reach,support)"],
                    "Instacart":["(use,instacart)","(use,service)","(never,use)"],
                    "Orders":["(place,order)","(cancel,order)","(get,order)","(receive,order)","(wrong,order)","(get,groceries)","(order,cancel)","(order,never)","(miss,items)","(order,place)","(order,get)","(items,order)","(miss,order)","(order,receive)","(order,wrong)","(issue,order)","(entire,order)","(pick,order)","(say,order)","(order,miss)","(order,arrive)","(original,order)","(find,order)","(order,time)","(order,items)","(make,order)","(order,take)","(order,say)","(complete,order)","(help,order)","(wait,order)","(order,leave)","(order,come)","(problem,order)","(another,order)","(put,order)","(correct,order)","(take,order)","(leave,order)","(charge,order)","(want,order)","(item,order)","(order,make)","(order,late)","(order,customer)","(order,pick)","(never,order)","(replace,order)","(try,order)","(whole,order)","(order,wasnt)","(order,complete)","(order,send)","(order,ready)","(give,wrong)","(order,correct)","(order,pay)","(see,order)","(order,great)","(order,another)","(change,order)","(order,without)","(order,since)","(back,order)","(call,order)","(account,order)","(miles,away)","(could,order)","(add,order)","(elses,order)","(half,order)","(bring,order)","(right,order)","(assign,order)","(order,right)","(order,next)","(drop,order)","(give,order)","(order,already)","(order,offer)","(late,order)","(order,drop)","(order,cant)","(send,order)","(ask,order)","(order,process)","(order,spend)","(order,check)","(order,system)","(fix,order)","(order,completely)","(multiple,order)","(order,lose)","(accept,order)","(order,driver)","(order,resolve)","(customer,order)","(confirm,order)","(incorrect,order)","(process,order)","(order,change)","(order,nothing)","(twice,order)","(order,confirm)","(order,mess)","(start,order)","(cant,order)","(order,incorrect)","(let,order)","(mess,order)","(customers,order)","(mark,order)","(full,order)"],
                    "Orders-Delivery":["(order,deliver)","(deliver,wrong)","(wrong,address)","(delivery,time)","(deliver,order)","(wrong,house)","(free,delivery)","(delivery,fee)","(never,deliver)","(get,delivery)","(order,delivery)","(items,deliver)","(delivery,service)","(delivery,order)","(get,deliver)","(correct,address)","(delivery,driver)","(suppose,deliver)","(say,deliver)","(time,delivery)","(change,delivery)","(deliver,items)","(food,deliver)","(deliver,time)","(hour,delivery)","(charge,delivery)","(deliver,correct)","(deliver,food)","(delivery,charge)","(could,deliver)","(deliver,right)","(delivery,address)","(delivery,guy)","(pay,delivery)","(deliver,address)","(issue,delivery)","(right,address)","(first,delivery)","(go,deliver)","(delivery,instructions)","(delivery,make)","(delivery,issue)","(delivery,get)","(still,deliver)","(delivery,deliver)","(deliver,get)","(late,delivery)","(actually,deliver)","(deliver,customer)","(refund,delivery)","(food,delivery)","(address,order)","(time,deliver)","(pick,delivery)","(problem,address)","(address,delivery)","(deliver,deliver)","(delivery,cancel)","(delivery,experience)","(another,address)","(delivery,take)","(didnt,deliver)","(address,wrong)"],
                    "Orders-Store":["(miss,item)","(store,close)","(order,store)","(store,order)","(fulfill,order)","(part,order)"],
                    "Orders-Shopper":["(delivery,person)","(driver,deliver)","(follow,instructions)","(person,deliver)","(delivery,people)","(check,order)","(another,delivery)","(follow,directions)","(delivery,drivers)","(get,better)","(sure,order)"],
                    "Time":["(take,long)","(waste,time)","(long,time)","(time,order)","(order,delay)","(two,hours)","(hours,later)","(wait,hours)","(multiple,time)","(within,hours)","(wait,minutes)","(hours,late)","(many,time)","(wait,time)","(order,hours)","(business,days)","(take,hours)","(half,hour)","(hold,minutes)","(much,time)","(long,wait)","(hours,order)","(days,ago)","(days,get)","(minutes,later)","(order,wait)","(take,hour)","(hours,get)","(almost,hours)","(hour,half)","(time,wait)","(time,give)","(pick,time)","(minutes,get)","(hour,later)","(time,receive)","(time,spend)","(time,waste)","(almost,hour)","(hour,time)","(two,weeks)","(hour,late)","(one,hour)","(hour,get)","(waste,hour)","(wait,hour)","(wait,min)","(minutes,wait)","(order,hour)","(wait,almost)","(hour,wait)","(faster,response)"],
                    "Zendesk":["(service,zendesk)","(helpful,zendesk)","(thank,zendesk)","(help,zendesk)","(zendesk,great)","(zendesk,excellent)","(issue,zendesk)","(zendesk,helpful)","(zendesk,issue)","(zendesk,support)","(resolve,zendesk)","(chat,zendesk)","(zendesk,resolution)"]
                    }

################### FUNCION PARA CATEGORIZAR ############################################
def get_category(x):
    key_list = []
    for key in Categories_brand.keys():
        for value in Categories_brand[key]:
            if value in x:
                if key not in key_list:
                    key_list.append(key)
    return(key_list)

################### FUNCION PARA RETORNAR EL BIGRAMA ##################################3

def get_bigram_key(x):
    key_list = []
    for value in Categories_brand[x["Category"]]:
        if value in x["bigrams"]:
            if value not in key_list:
                key_list.append(value)
    return(key_list)

# Se aplica la funcion al df:
df2["Category"]=df2["bigrams"].apply(lambda x: get_category(x))
df2["Category"]=df2["Category"].map(str)

df2=df2[df2["Category"]!="[]"]
df2["Category"]=df2["Category"].map(str).apply(lambda x: x.replace("[",""))
df2["Category"]=df2["Category"].map(str).apply(lambda x: x.replace("]",""))
df2["Category"]=df2["Category"].map(str).apply(lambda x: x.replace("'",""))
df2["Category"]=df2["Category"].map(str).apply(lambda x: x.strip())
df2["Category"]=df2["Category"].map(str)
df2.Category = df2.Category.str.split(",")
df2 = df2.explode('Category')
df2["Category"]=df2["Category"].map(str).apply(lambda x: x.strip())
#Obtener bigramas por categoria: 
df2["Category2"]=df2.apply(get_bigram_key, axis=1)
df2["Category2"]=df2["Category2"].map(str).apply(lambda x: x.replace("[",""))
df2["Category2"]=df2["Category2"].map(str).apply(lambda x: x.replace("]",""))
df2["Category2"]=df2["Category2"].map(str).apply(lambda x: x.replace("'",""))
df2["Category2"]=df2["Category2"].map(str).apply(lambda x: x.replace("), (","-"))
df2["Category2"]=df2["Category2"].map(str)
df2.Category2 = df2.Category2.str.split("-")
df2 = df2.explode('Category2')
df2["Category2"]=df2["Category2"].map(str).apply(lambda x: x.replace("(",""))
df2["Category2"]=df2["Category2"].map(str).apply(lambda x: x.replace(")",""))
df2["Category2"]=df2["Category2"].map(str).apply(lambda x: x.replace(","," "))
df2["Category2"]=df2["Category2"].map(str).apply(lambda x: x.strip())
df2=df2.drop("bigrams", axis=1)
df2["Category1"]="Brand"
df2.to_csv(r"C:\Users\bustossanchez.9\OneDrive - Teleperformance\Procesos\Instacard\RawData\Categorization_Brand.csv")

In [13]:
#####CATEGORIZATION CHANNEL ################
Categories_channel = {"Phone":["(phone,number)","(phone,call)","(speak,someone)","(get,phone)","(receive,call)","(someone,call)","(person,talk)","(phone,support)","(talk,someone)","(pick,phone)","(get,call)","(hang,phone)","(need,speak)","(one,call)","(time,phone)","(support,phone)","(please,call)","(call,chat)","(live,person)","(never,hear)"],
                        "Chat":["(chat,support)","(chat,get)","(online,chat)","(time,chat)"],
                        "Email":["(get,email)","(via,email)","(email,send)"]}

################### FUNCION PARA CATEGORIZAR ############################################
def get_category(x):
    key_list = []
    for key in Categories_channel.keys():
        for value in Categories_channel[key]:
            if value in x:
                if key not in key_list:
                    key_list.append(key)
    return(key_list)

################### FUNCION PARA RETORNAR EL BIGRAMA ##################################3

def get_bigram_key(x):
    key_list = []
    for value in Categories_channel[x["Category"]]:
        if value in x["bigrams"]:
            if value not in key_list:
                key_list.append(value)
    return(key_list)

# Se aplica la funcion al df:
df3["Category"]=df3["bigrams"].apply(lambda x: get_category(x))
df3["Category"]=df3["Category"].map(str)

df3=df3[df3["Category"]!="[]"]
df3["Category"]=df3["Category"].map(str).apply(lambda x: x.replace("[",""))
df3["Category"]=df3["Category"].map(str).apply(lambda x: x.replace("]",""))
df3["Category"]=df3["Category"].map(str).apply(lambda x: x.replace("'",""))
df3["Category"]=df3["Category"].map(str).apply(lambda x: x.strip())
df3["Category"]=df3["Category"].map(str)
df3.Category = df3.Category.str.split(",")
df3 = df3.explode('Category')
df3["Category"]=df3["Category"].map(str).apply(lambda x: x.strip())
#Obtener bigramas por categoria: 
df3["Category2"]=df3.apply(get_bigram_key, axis=1)
df3["Category2"]=df3["Category2"].map(str).apply(lambda x: x.replace("[",""))
df3["Category2"]=df3["Category2"].map(str).apply(lambda x: x.replace("]",""))
df3["Category2"]=df3["Category2"].map(str).apply(lambda x: x.replace("'",""))
df3["Category2"]=df3["Category2"].map(str).apply(lambda x: x.replace("), (","-"))
df3["Category2"]=df3["Category2"].map(str)
df3.Category2 = df3.Category2.str.split("-")
df3 = df3.explode('Category2')
df3["Category2"]=df3["Category2"].map(str).apply(lambda x: x.replace("(",""))
df3["Category2"]=df3["Category2"].map(str).apply(lambda x: x.replace(")",""))
df3["Category2"]=df3["Category2"].map(str).apply(lambda x: x.replace(","," "))
df3["Category2"]=df3["Category2"].map(str).apply(lambda x: x.strip())
df3=df3.drop("bigrams", axis=1)
df3["Category1"]="Channel"
df3.to_csv(r"C:\Users\bustossanchez.9\OneDrive - Teleperformance\Procesos\Instacard\RawData\Categorization_Channel.csv")

In [14]:
####CONCATENADOR###############
dfa=pd.read_csv(r"C:\Users\bustossanchez.9\OneDrive - Teleperformance\Procesos\Instacard\RawData\Categorization_Agent.csv")
dfb=pd.read_csv(r"C:\Users\bustossanchez.9\OneDrive - Teleperformance\Procesos\Instacard\RawData\Categorization_Brand.csv")
dfc=pd.read_csv(r"C:\Users\bustossanchez.9\OneDrive - Teleperformance\Procesos\Instacard\RawData\Categorization_Channel.csv")

conc=[dfa,dfb,dfc]

df_final=pd.concat(conc)
df_final=df_final.set_index("Date")

pol= lambda x: TextBlob(x).sentiment.polarity

df_final["Polarity"]=df_final['Feedback'].apply(pol)

df_final.head()


,Agent,EMAIL,Type,Channel,CR,Menu_Path,CSAT,Feedback,TOUCH,WAREHOUSE,Category,Category2,Category1,Polarity
Date,,,,,,,,,,,,,,
2023-06-21,Silvia Ojeda Vizcarra,silvia.ojedavizcarra@teleperformance.com,Customers,Chat,Entire Delivery Missing,FDO,Bad,have representatives who can quickly identify ...,2,Kroger Delivery Now,Issue Resolution,resolve issue,Agent,0.016667
2023-06-21,Silvia Ojeda Vizcarra,silvia.ojedavizcarra@teleperformance.com,Customers,Chat,Entire Delivery Missing,FDO,Bad,have representatives who can quickly identify ...,2,Kroger Delivery Now,Issue Resolution,issue take,Agent,0.016667
2023-06-21,Harumi Rocio Villanueva Navarro,harumi.villanuevanavarro@teleperformance.com,Customers,Phone,Missing Items,T1,Bad,still waiting to receive an email receipt for ...,1,Sprouts Farmers Market,Issue Resolution,still wait,Agent,-0.133333
2023-06-21,Carla Soller,carla.sollersedano@teleperformance.com,Customers,Phone,Missing Items,T1,Good,instacart has the very best customer service ...,1,Food Lion,Issue Resolution,problem solve,Agent,0.529167
2023-06-21,Edith Cielo Changa Rojas,edith.changarojas@teleperformance.co,Customers,Chat,Wrong Order,FDO,Bad,i didnt get to respond,1,Market Basket,Issue Resolution,didnt get,Agent,0.000000


In [15]:
df_final.to_csv(r"C:\Users\bustossanchez.9\OneDrive - Teleperformance\Procesos\Instacard\DataPBI\Categorization.csv")

In [16]:
#from datetime import datetime as dt
print("done", dt.now()) 

done 2023-06-22 15:34:44.653749


In [17]:
#import pymssql
#import pyodbc
#import pandas as pd
#
#conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
#                     'Server= TPCCP-DB07\SCOFFS;'
#                     'Database=Bancolombia;'
#                     'Trusted_Connection=yes;') 
#
#data ='''SELECT * FROM [Bancolombia].[dbo].[tbNequiReiteratividad] WITH(NOLOCK)'''
#
#BDInfinite=pd.read_sql(sql=data,con=conn)
#df=BDInfinite
#
#df